In [20]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

все слова с учетом регистра<br>
все в одном регистре<br>
без стоп слов<br>
выделение специфичных для классов<br>
-- -- 
лог регрессия, случайный лес, байес

In [21]:
ord('Н')%3

0

In [22]:
def fitpredict(xtrain,ytrain,xtest,ytest):
    
    lr =  lm.LogisticRegression()
    rfc = en.RandomForestClassifier()
    cnb = nb.ComplementNB()

    lr.fit(xtrain,ytrain)
    rfc.fit(xtrain,ytrain)
    cnb.fit(xtrain,ytrain)

    pred_lr = lr.predict(xtest)
    pred_rfc = rfc.predict(xtest)
    pred_cnb = cnb.predict(xtest)
    
    print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
        'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
        'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))
    
    return lr, rfc, cnb

In [23]:
corpus = pd.read_csv('SMSpam.csv')

In [24]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['Message']:
    cnt += len(i.split(' '))
for i in corpus[corpus['Category']=='spam']['Message']:
    cntS += len(i.split(' '))
for i in corpus[corpus['Category']=='ham']['Message']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['Category'].value_counts()['spam'], 'spam и',corpus['Category'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5572 
Из них 747 spam и 4825 ham 
Всего слов:  87265 
из них 17788 spam и 69477 ham


In [25]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['Message'],corpus['Category'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4457,) 
Тестовая выборка (1115,)


-- -- 
Используем все слова с учетом регистра:

In [26]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(xtrain)
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 9666 токенов


In [27]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97840   0.99895   0.98857       952
        spam    0.99301   0.87117   0.92810       163

    accuracy                        0.98027      1115
   macro avg    0.98570   0.93506   0.95834      1115
weighted avg    0.98053   0.98027   0.97973      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97143   1.00000   0.98551       952
        spam    1.00000   0.82822   0.90604       163

    accuracy                        0.97489      1115
   macro avg    0.98571   0.91411   0.94577      1115
weighted avg    0.97561   0.97489   0.97389      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98629   0.98214   0.98421       952
        spam    0.89820   0.92025   0.90909       163

    accuracy                        0.97309      1115
   macro avg    0.94225   0.95119   0.94665      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем все слова, но в одном регистре:

In [28]:
cv = tx.CountVectorizer()
cv.fit(xtrain)
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 7762 токенов


In [29]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97837   0.99790   0.98804       952
        spam    0.98611   0.87117   0.92508       163

    accuracy                        0.97937      1115
   macro avg    0.98224   0.93453   0.95656      1115
weighted avg    0.97950   0.97937   0.97884      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97143   1.00000   0.98551       952
        spam    1.00000   0.82822   0.90604       163

    accuracy                        0.97489      1115
   macro avg    0.98571   0.91411   0.94577      1115
weighted avg    0.97561   0.97489   0.97389      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98734   0.98319   0.98526       952
        spam    0.90419   0.92638   0.91515       163

    accuracy                        0.97489      1115
   macro avg    0.94577   0.95479   0.95021      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Найдем все стоп-слова, которые встречаются в большом количестве сообщений (более 500 , что около 9% всей выборки) и уберем их:

In [30]:
cv = tx.CountVectorizer(max_df = 500)
cv.fit(xtrain)
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(xtrain)
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 8 токенов
{'my', 'me', 'in', 'is', 'you', 'the', 'and', 'to'}


In [31]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98142   0.99895   0.99011       952
        spam    0.99315   0.88957   0.93851       163

    accuracy                        0.98296      1115
   macro avg    0.98729   0.94426   0.96431      1115
weighted avg    0.98314   0.98296   0.98257      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97140   0.99895   0.98498       952
        spam    0.99265   0.82822   0.90301       163

    accuracy                        0.97399      1115
   macro avg    0.98202   0.91359   0.94400      1115
weighted avg    0.97451   0.97399   0.97300      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98624   0.97899   0.98260       952
        spam    0.88235   0.92025   0.90090       163

    accuracy                        0.97040      1115
   macro avg    0.93430   0.94962   0.94175      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки) и уберем:

In [32]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(xtrain)
stop = cv.stop_words_
print('Всего', len(stop), 'токенов отмечены как стоп-слова.')
#print(stop) // не будем выводить список слов, так как он слишком большой

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(xtrain)
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
print('Работаем с {0} - {1} = {2}'.format(len(cv.vocabulary_) + len(stop), len(stop), len(cv.vocabulary_)), 'токенами.')

Всего 6906 токенов отмечены как стоп-слова.
Работаем с 7762 - 6906 = 856 токенами.


In [33]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98240   0.99685   0.98957       952
        spam    0.97987   0.89571   0.93590       163

    accuracy                        0.98206      1115
   macro avg    0.98113   0.94628   0.96273      1115
weighted avg    0.98203   0.98206   0.98173      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97639   0.99895   0.98754       952
        spam    0.99291   0.85890   0.92105       163

    accuracy                        0.97848      1115
   macro avg    0.98465   0.92892   0.95430      1115
weighted avg    0.97880   0.97848   0.97782      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98816   0.96429   0.97608       952
        spam    0.81720   0.93252   0.87106       163

    accuracy                        0.95964      1115
   macro avg    0.90268   0.94840   0.92357      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Используем встроенный список стоп-слов:

In [34]:
cv = tx.CountVectorizer(stop_words = 'english')
cv.fit(xtrain)
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97641   1.00000   0.98806       952
        spam    1.00000   0.85890   0.92409       163

    accuracy                        0.97937      1115
   macro avg    0.98821   0.92945   0.95608      1115
weighted avg    0.97986   0.97937   0.97871      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97140   0.99895   0.98498       952
        spam    0.99265   0.82822   0.90301       163

    accuracy                        0.97399      1115
   macro avg    0.98202   0.91359   0.94400      1115
weighted avg    0.97451   0.97399   0.97300      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99237   0.95693   0.97433       952
        spam    0.79188   0.95706   0.86667       163

    accuracy                        0.95695      1115
   macro avg    0.89213   0.95699   0.92050      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем только слова с высокой важностью:

In [35]:
cv = tx.CountVectorizer()
cv.fit(xtrain)
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

e = en.RandomForestClassifier()
e.fit(tr_xtrain,ytrain)
e.score(tr_xtest,ytest)

0.9739910313901345

In [36]:
imp = pd.DataFrame({'name' : cv.vocabulary_.keys(), 'importance' : e.feature_importances_}) 
imp = imp.sort_values(by = 'importance', ascending = False)
top100 = imp.head(n=1000)['name'].to_numpy()

cv = tx.CountVectorizer(vocabulary = top100)
cv.fit(xtrain)
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.95762   0.99685   0.97684       952
        spam    0.97581   0.74233   0.84321       163

    accuracy                        0.95964      1115
   macro avg    0.96671   0.86959   0.91002      1115
weighted avg    0.96028   0.95964   0.95730      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96633   0.99475   0.98033       952
        spam    0.96296   0.79755   0.87248       163

    accuracy                        0.96592      1115
   macro avg    0.96464   0.89615   0.92641      1115
weighted avg    0.96583   0.96592   0.96457      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.97730   0.94958   0.96324       952
        spam    0.74737   0.87117   0.80453       163

    accuracy                        0.93812      1115
   macro avg    0.86233   0.91037   0.88389      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем tf-idf векторизатор слов в одном регистре:

In [37]:
tfidfv = tx.TfidfVectorizer()
tfidfv.fit(corpus['Message'])
tr_xtrain = tfidfv.transform(xtrain)
tr_xtest = tfidfv.transform(xtest)

In [38]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.95010   1.00000   0.97441       952
        spam    1.00000   0.69325   0.81884       163

    accuracy                        0.95516      1115
   macro avg    0.97505   0.84663   0.89663      1115
weighted avg    0.95739   0.95516   0.95167      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97044   1.00000   0.98500       952
        spam    1.00000   0.82209   0.90236       163

    accuracy                        0.97399      1115
   macro avg    0.98522   0.91104   0.94368      1115
weighted avg    0.97476   0.97399   0.97292      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98512   0.97374   0.97940       952
        spam    0.85632   0.91411   0.88427       163

    accuracy                        0.96502      1115
   macro avg    0.92072   0.94392   0.93184      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())